<a href="https://colab.research.google.com/github/marekpiotradamczyk/ml_uwr_22/blob/main/kmeans_deep_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How it differs from the default solution? 

More smaller patches and denser patches extraction from images.
Note that it takes significant time to execute all steps

In [10]:
C          = 50000 # 1000 - 0.24, 5000 - 0.3432, 25000 - 0.3861
PATCH_SIZE = 4
patch_num  = 100000
STRIDE     = 4
k          = 64

# Load data & default imports

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import scipy.stats as sstats
import multiprocessing as mp
from sklearn import datasets
import sklearn.linear_model
from tqdm.auto import tqdm
from matplotlib import animation, pyplot, rc
import matplotlib.pyplot as plt
import httpimport
from os.path import join
import os.path
from PIL import Image

from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np

from sklearn.feature_extraction import image
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.linear_model import LogisticRegression


In [2]:
!pip install -q gdown httpimport
![ -e cifar.npz ] || gdown 'https://drive.google.com/uc?id=1oBzZdtg2zNTPGhbRy6DQ_wrf5L5OAhNR' -O cifar.npz

In [3]:
with np.load('cifar.npz') as data:
    cifar_train_data = data['train_data']
    cifar_train_labels = data['train_labels']
    cifar_test_data = data['test_data']
    cifar_test_labels = data['test_labels']


def exists(file_path):
    return os.path.isfile(file_path)

In [4]:
X_trn = cifar_train_data
y_trn = cifar_train_labels
X_tst = cifar_test_data
y_tst = cifar_test_labels

# All in one place

In [5]:
class clf():
      
    def __init__(self):
        pass
    
    def contrast(self, image):
        return (image-image.min())/(image.max() - image.min())

    def normalize_patch(self, patch, eps=10):
        return (patch - patch.mean())/np.sqrt(patch.var() + eps)

    def whiten(self, X):
        X_norm = (X - X.mean(axis=0))/X.std(axis=0)
        cov = np.cov(X_norm, rowvar=False) 
        U,S,V = np.linalg.svd(cov)

        X_zca = U.dot(np.diag(1.0/np.sqrt(S + 0.1))).dot(U.T).dot(X_norm.T).T
        return X_zca

    def centroids(self, data, k):
        kmeans = MiniBatchKMeans(n_clusters=k, random_state=0, verbose=False, n_init=1, max_iter=200, batch_size=10000)
        kmeans.fit(data)
        return kmeans.cluster_centers_
    
    def extract_patches(self, data):
        patches = []
        reshaped = data.reshape(-1,32,32,3)
        n = int(patch_num / (32-PATCH_SIZE+1) ** 2 + 1)
        for i in range(n):
            for r in range(32-PATCH_SIZE+1):
                for c in range(32-PATCH_SIZE+1):
                    patch = reshaped[i][c:(c+PATCH_SIZE),r:(r+PATCH_SIZE)].flatten()
                    patch_norm = self.normalize_patch(patch, eps=10)
                    patches.append(patch_norm)

        P = np.vstack(patches)
        return self.whiten(P)
    
    def dist(self, x,y):
        return np.sqrt((x - y).dot(x-y))

    def create_patch_features(self, X):    
        X_mapped_list_per_image = []
        for i in range(X.shape[0]):
            mapped_features = []
            for r in range(0, 32-PATCH_SIZE+1, STRIDE):
                for c in range(0, 32-PATCH_SIZE+1, STRIDE):
                    patch = X[i].reshape(32,32,3)[c:(c+PATCH_SIZE),r:(r+PATCH_SIZE)].flatten()
                    patch_norm = self.normalize_patch(patch, eps=0.01)
                    mapped_features.append([dist(patch_norm, f) for f in filters_final])
            X_mapped_list_per_image.append(np.vstack(mapped_features))
        X_mapped = np.asarray(X_mapped_list_per_image).reshape(-1, ((32-PATCH_SIZE)//STRIDE+1)**2*filters_final.shape[0])
        return X_mapped

    def create_patch_features__vectorized(self, X):    
        X_mapped_list_per_image = []
        for i in range(X.shape[0]):
            patches = image.extract_patches_2d(X[i], (PATCH_SIZE, PATCH_SIZE))
            strided_patches = patches.reshape( 32-PATCH_SIZE+1 , 32-PATCH_SIZE+1, PATCH_SIZE, PATCH_SIZE, 3)[::STRIDE,::STRIDE,:,:,:]
            strided_patches = strided_patches.reshape(((32-PATCH_SIZE)//STRIDE+1)**2, PATCH_SIZE * PATCH_SIZE * 3)
            mapped_features = euclidean_distances(np.asarray([self.normalize_patch(patch, eps=0.01) for patch in strided_patches]), self.filters_final)
            X_mapped_list_per_image.append(mapped_features.reshape(((32-PATCH_SIZE)//STRIDE+1)**2 * self.filters_final.shape[0]))
        X_mapped = np.asarray(X_mapped_list_per_image)
        return X_mapped

    def normalize(self, data):
        return (data - data.mean(axis=0))/data.std(axis=0)  
 
    
    def fit(self, X_trn, y_trn, k):
        P_zca = self.extract_patches(X_trn)
        self.filters_final = self.centroids(P_zca,k)
        X_mapped_trn = self.create_patch_features__vectorized(X_trn)
        X_mapped_trn_norm = self.normalize(X_mapped_trn)
        self.model = LogisticRegression(random_state=0, max_iter=80, n_jobs=-1, verbose=False)
        self.model.fit(X_mapped_trn_norm, y_trn.flatten());
        
        
    def accuracy(self, X_tst, y_tst):
        X_mapped_tst = self.create_patch_features__vectorized(X_tst)
        X_mapped_tst_norm = self.normalize(X_mapped_tst)
        y_tst_pred = self.model.predict(X_mapped_tst_norm)
        return (y_tst.flatten() == y_tst_pred).mean()        

# testing

In [11]:
model = clf()
model.fit(X_trn[:C], y_trn[:C], k) # todo make k global
print(model.accuracy(X_tst[:C], y_tst[:C]))

/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.4938


# Finding optimal k

In [ ]:
C          = 5000
PATCH_SIZE = 6
patch_num  = 100000
STRIDE     = 8

errs_k = {}

for kroot in tqdm(range(2,32)):
    k = kroot **2
    model = clf()
    model.fit(X_trn[:C], y_trn[:C], k)
    errs_k[k] = model.accuracy(X_tst[:C], y_tst[:C])
    


In [ ]:
plt.plot(errs_k.keys(), errs_k.values())
errs_k

# Finding optimal patch size

In [ ]:
C          = 5000
patch_num  = 100000
STRIDE     = 8

errs_patch_size = {}

for i in tqdm(range(2,30)):
    PATCH_SIZE = i
    model = clf()
    model.fit(X_trn[:C], y_trn[:C], k)
    errs_patch_size[PATCH_SIZE] = model.accuracy(X_tst[:C], y_tst[:C])

In [ ]:
plt.plot(errs_patch_size.keys(), errs_patch_size.values())
errs_patch_size

# Find optimal stride

In [ ]:
C          = 5000
patch_num  = 100000
PATCH_SIZE = 4

errs_stride = {}

for i in tqdm(range(2,30)):
    STRIDE = i
    model = clf()
    model.fit(X_trn[:C], y_trn[:C], 9)
    errs_stride[STRIDE] = model.accuracy(X_tst[:C], y_tst[:C])

In [ ]:
plt.plot(errs_stride.keys(), errs_stride.values())
errs_stride